In [1]:
import json
import pandas as pd
import numpy as np
import mysql.connector

In [2]:
def fetch_all(query):
    connection = mysql.connector.connect(host='localhost', database='rncs', user='admin', password='Pg49mkMfF4SuLLbA')
    cursor = connection.cursor(buffered=True)    
    cursor.execute(query)
    full = cursor.fetchall()
    cursor.close()
    connection.close()
    return full

# Extract basic info

* Legal form OK
* Main activity OK
* Employer or not
* Capitalisation OK
* City of main establishment OK
* ZIP code (for region) OK
* Number of owners
* Country of domicile of owners

In [3]:
sirens = np.load('/project/0_cleaning/output_cleaning/sirens.npy').tolist()
len(sirens)

3573

In [4]:
query = f'''
    SELECT
        insee_unite_legale.siren,
        insee_unite_legale.date_creation,
        insee_unite_legale.categorie_entreprise,
        insee_unite_legale.activite_principale,
        insee_unite_legale.caractere_employeur,      
        insee_etablissement.code_postal,
        insee_etablissement.libelle_commune,
        imr_pm.forme_juridique,
        imr_pm.type_capital,
        imr_pm.devise,
        imr_pm.capital,
        imr_pm.capital_actuel
    FROM
        insee_unite_legale
    LEFT JOIN imr_pm ON insee_unite_legale.siren = imr_pm.siren
    LEFT JOIN insee_etablissement ON insee_unite_legale.siren = insee_etablissement.siren
    WHERE
        insee_unite_legale.siren IN {tuple(sirens)} AND
        insee_etablissement.etablissement_siege = 'true' AND
        imr_pm.libelle_evt <> 'Modifications relatives au dossier' AND
        imr_pm.type_inscription = 'P'
    
    UNION

    SELECT
        imr_pp.siren,
        imr_pp.date_immatriculation,
        'PP',
        '',
        'N',       
        imr_pp.code_postal,
        imr_pp.ville,
        'Entreprise individuelle',
        'EI',
        '',
        '',
        ''
    FROM
        imr_pp
    WHERE
        imr_pp.siren IN {tuple(sirens)} AND
        imr_pp.libelle_evt <> 'Modifications relatives au dossier' AND
        imr_pp.type_inscription = 'P'
    
'''

result = fetch_all(query)
result_df = pd.DataFrame(result, 
                         columns=['siren', 'creation_date', 'size', 'main_activity', 'employer', 'zip', 'city',
                                  'legal_form', 'type_capital', 'ccy_capital', 'capital', 'current_capital']).drop_duplicates()

result_df.head()

,siren,creation_date,size,main_activity,employer,zip,city,legal_form,type_capital,ccy_capital,capital,current_capital
0,006580195,1965-01-01,PME,29.10Z,O,44600,SAINT NAZAIRE,Société par actions simplifiée,F,Euros,12474989.36,
1,007220338,1964-01-01,PME,46.49Z,O,80150,LE BOISLE,Société par actions simplifiée,F,Euros,37000.0,
2,015751530,1957-01-01,PME,10.72Z,O,21000,DIJON,Société anonyme à conseil d'administration,F,Euros,120000.0,
3,057813677,1957-01-01,PME,47.52A,O,13001,MARSEILLE 1,Société par actions simplifiée,F,Euros,42000.0,
4,065501850,1965-01-01,PME,47.78B,O,38350,LA MURE,Société par actions simplifiée,F,Euros,100000.0,


In [40]:
result_df['siren'].isna().sum()

0

# Ensure that len = len(siren)

In [5]:
def to_float(x):
    if x == '':
        x = np.nan
    else:
        x = float(x)
    return x

In [6]:
def dedup_result_df(result_df, sirens):
    
    result_df['creation_date'] = pd.to_datetime(result_df['creation_date'])
    result_df['capital'] = result_df['capital'].apply(to_float)
    result_df['current_capital'] = result_df['current_capital'].apply(to_float)
    result_df.replace({'ccy_capital': {
                                        'Euros':'EUR',
                                        'EUROS':'EUR',
                                        'Franc suisse': 'CHF',
                                        'LIVRE STERLING': 'GBP',
                                        'SHEKEL': 'ILS',
                                        'YEN': 'JPY'}}, inplace=True)

    dedup_df = pd.DataFrame(columns=result_df.columns)

    for siren in sirens:

        df = result_df[result_df['siren']==siren].drop_duplicates()

        if df.shape[0]>1:
            df = df[df['creation_date']==max(df['creation_date'])]
            if df.shape[0]>1:
                df = df[df['capital']==np.nanmax(df['capital'])]
                if df.shape[0]>1:
                    df = df[df['legal_form']==max(df['legal_form'])] # data quality issue, have to pick either one of the record
        elif df.shape[0]==0:
            df = pd.DataFrame(columns=result_df.columns)
            df.loc[0, 'siren'] = siren

        dedup_df = pd.concat([dedup_df,df]).reset_index(drop=True)

    return dedup_df

In [7]:
dedup_df = dedup_result_df(result_df, sirens)
print(len(dedup_df['siren'].unique()))
print(dedup_df.shape)

3573
(3573, 12)


In [8]:
dedup_df['ccy_capital'].value_counts().head()

EUR    3063
        490
JPY       2
CHF       1
ILS       1
Name: ccy_capital, dtype: int64

In [9]:
dedup_df[dedup_df['siren']=='823268677']

,siren,creation_date,size,main_activity,employer,zip,city,legal_form,type_capital,ccy_capital,capital,current_capital
1028,823268677,2016-10-11,PME,47.78C,N,75017,PARIS 17,Société par actions simplifiée,F,EUR,2162.0,NaN


In [41]:
dedup_df['siren'].isna().sum()

0

# Cleaning extracted data

In [46]:
clean_df = dedup_df.copy()
clean_df.head(1)

,siren,creation_date,size,main_activity,employer,zip,city,legal_form,type_capital,ccy_capital,capital,current_capital
0,332224393,1985-05-01,PME,10.13B,O,59640,DUNKERQUE,Société à responsabilité limitée,F,EUR,7622.45,NaN


In [47]:
clean_df.replace({
    'size': {'':None, np.nan: None},
    'zip': {'':None, np.nan: None},
    'city': {'':None, np.nan: None},
    'main_activity': {'':None, np.nan: None},
    'type_capital': {'':None, np.nan: None},
    'ccy_capital': {'':None, np.nan: None},
    'creation_date': {np.datetime64('1900-01-01'): pd.NaT}
}, inplace=True)
clean_df.shape

(3573, 12)

In [52]:
def clean_legal_form(clean_df):
    mapping = {
        'Société par actions simplifiée': [
            'Société par actions simplifiée',
            'Société par actions simplifiée  à associé unique et capital variable',
            'Société par actions simplifiée à associé unique',
            'Société par actions simplifiée à capital variable'],
        'Société anonyme': [
            "Société anonyme à conseil d'administration",
            'Société anonyme',
            'Société anonyme à directoire et conseil de surveillance',
            "Société anonyme d'économie mixte",
            'Société anonyme sportive professionnelle'],
        'Société à responsabilité limitée': [
            'Société à responsabilité limitée',
            'Société à responsabilité limitée à associé unique',
            'Société à responsabilité limitée à capital variable',
            'Société à responsabilité limitée de presse'],
         'Société coopérative': [
             'Société coopérative agricole à capital variable',
             'Société coopérative à forme anonyme',
             'Société coopérative agricole',
             'Société coopérative à capital variable à capital variable à capital variable',
             'Société coopérative de production à responsabilité limitée à capital variable',
             'Société coopérative de consommation à forme anonyme',
             'Société coopérative ouvrière de production à responsabilité limitée',
             "Société coopérative d'intérêt collectif à responsabilité limitée à capital variable",
             'Société coopérative à responsabilité limitée',
             'Société coopérative ouvrière de production à forme anonyme',
             'Société coopérative de production à forme anonyme et capital variable',
             'Société coopérative de production à responsabilité limitée',
             'Société coopérative de production à forme anonyme',
             'Société coopérative à responsabilité limitée à capital variable',
             'Société coopérative ouvrière de production à responsabilité limitée à capital variable',
             'Société coopérative à forme anonyme à capital variable',
             "Société coopérative d'intérêt collectif à responsabilité limitée",
             'Société coopérative à capital variable'],
        'Société civile': [
            'Société civile',
            "Société civile d'intérêt collectif agricole",
            "Société civile d'exploitation agricole",
            'Société civile professionnelle',
            'Société civile immobilière'],
        "Société d'exercice libéral": [
            "Société d'exercice libéral à associé unique",
            "Société d'exercice libéral à responsabilité limitée",
            "Société d'exercice libéral à responsabilité limitée à associé unique"],
        'Société de droit étranger': [
            'Société de droit étranger',
            "Société anonyme d'un Etat non membre de la CE ou non partie à l'accord sur l'Espace économique européen"],
        'Groupement': [
            "Groupement d'intérêt économique",
            "Groupement agricole d'exploitation en commun"],
        'Etablissement public' : ['Etablissement public à caractère industriel et commercial'],
        'Exploitation agricole' : ['Exploitation agricole à responsabilité limitée'],
        'Autre': [
            'Exploitation agricole',
            'Société civile',
            "Société d'exercice libéral",
            "Groupement",
            "Société en nom collectif",
            "Société en commandite simple",
            "Etablissement public"]
    }
    mapping_transformed = {v : k for k, V in mapping.items() for v in V}
    clean_df['legal_form'] = clean_df['legal_form'].replace(mapping_transformed, regex=True)

In [53]:
clean_legal_form(clean_df)
clean_df['legal_form'].value_counts()

Société à responsabilité limitée    1856
Société par actions simplifiée      1094
Entreprise individuelle              483
Société anonyme                       51
Autre                                 36
Société coopérative                   31
Société de droit étranger              7
Name: legal_form, dtype: int64

In [54]:
clean_df.shape

(3573, 12)

In [55]:
clean_df[clean_df['siren']== '794350942']

,siren,creation_date,size,main_activity,employer,zip,city,legal_form,type_capital,ccy_capital,capital,current_capital
358,794350942,2014-07-30,PP,None,N,20290,Borgo,Entreprise individuelle,EI,None,NaN,NaN


In [56]:
clean_df['siren'].isna().sum()

0

# Extract features
## Departement and region

In [68]:
features_df = clean_df.copy()
features_df.shape

(3573, 12)

In [69]:
def extract_dpt(x):
    if x is None:
        return x
    else:
        return int(str(x)[:2])
features_df['zip'] = features_df['zip'].apply(extract_dpt)
print(features_df['zip'].unique())

[59. 75. 77. 38. 35. 42. nan 93. 13. 69. 34. 87. 14. 31. 47. 62. 76. 74.
 26. 33. 63. 79. 92. 60. 80. 95. 17.  6. 83. 19. 37. 29. 51. 82. 48. 94.
 45. 44. 71. 39. 43. 58. 11. 25. 50. 81. 84. 21. 66. 40. 55. 49. 70. 64.
  2. 30. 10. 91. 78. 54. 32. 85. 27.  8. 73. 15. 53. 52. 56. 65.  7. 72.
 20. 86.  1.  5. 46.  4. 18. 24. 22.  3. 12. 61. 68. 90. 89. 41. 28. 88.
 16.  9. 36. 23.]


In [70]:
zips = pd.read_csv('french-zips.csv', sep=';')
zips.head()

,code,sub-region,region
0,1,Ain,Auvergne-Rhône-Alpes
1,2,Aisne,Hauts-de-France
2,3,Allier,Auvergne-Rhône-Alpes
3,4,Alpes-de-Haute-Provence,Provence-Alpes-Côte d'Azur
4,5,Hautes-Alpes,Provence-Alpes-Côte d'Azur


In [71]:
features_df = features_df.merge(zips, how='left', left_on='zip', right_on='code')
features_df.drop(columns=['code'], inplace=True)
features_df.shape

(3573, 14)

In [72]:
features_df['siren'].isna().sum()

0

## Activity

In [73]:
def extract_act(x):
    if x is None:
        return np.nan
    else:
        return float(str(x).split('.')[0])
features_df['main_activity'] = features_df['main_activity'].apply(extract_act)
features_df.loc[features_df['main_activity'] == 0.0, 'main_activity'] = np.nan
print(features_df['main_activity'].unique())

[10. 47. 46. 85. nan 15. 71. 11. 58. 77. 81. 74. 43. 93. 70. 90. 64. 62.
 20. 53. 56. 32. 45. 95. 14. 23. 33. 96. 31. 27. 29. 55. 91. 82. 28. 26.
 73. 59. 13. 18. 30.  1. 63. 79. 38. 69. 68. 42. 16. 65. 25.  3. 80. 88.
 17. 51. 49.  2. 22. 72. 86. 61. 52. 87. 66. 41. 84. 21.]


In [74]:
activities = pd.read_csv('naf2008_liste_n2.csv', sep=';')
activities.head()

,code,activity_class
0,1,"Culture et production animale, chasse et servi..."
1,2,Sylviculture et exploitation forestière
2,3,Pêche et aquaculture
3,5,Extraction de houille et de lignite
4,6,Extraction d'hydrocarbures


In [75]:
features_df = features_df.merge(activities, how='left', left_on='main_activity', right_on='code')
features_df.drop(columns=['code'], inplace=True)
features_df.shape

(3573, 15)

In [76]:
features_df['siren'].isna().sum()

0

## Age

In [77]:
features_df['years_since_creation'] = 2019 - features_df['creation_date'].dt.year
features_df['years_since_creation'].value_counts().head()

4.0    352
3.0    314
6.0    234
5.0    215
8.0    204
Name: years_since_creation, dtype: int64

In [78]:
features_df.shape

(3573, 16)

## Capital

In [79]:
def get_capital(df):
    capital_final = []
    capital = features_df['capital'].tolist()
    current_capital = features_df['current_capital'].tolist()
    for i in range(len(capital)):
        if np.isnan(capital[i]):
            if np.isnan(current_capital[i]):
                capital_final.append(np.nan)
            else:
                capital_final.append(current_capital[i])
        else:
            if np.isnan(current_capital[i]):
                capital_final.append(capital[i])
            else:
                capital_final.append(max(capital[i], current_capital[i]))
    
    return capital_final

features_df['amount_capital'] = get_capital(features_df)

## Employer capacity

In [80]:
features_df['employer'] = [int(x=='O') for x in features_df['employer']]

In [81]:
features_df.shape

(3573, 17)

## Personality type

In [82]:
features_df['personality_type'] = [int(x=='PP') for x in features_df['size']]
features_df.shape

(3573, 18)

## Drop columns no longer needed

In [83]:
features_df.drop(columns=['size', 'creation_date', 'main_activity', 'capital', 'current_capital', 'zip', 'city'], inplace=True)
features_df.head()

,siren,employer,legal_form,type_capital,ccy_capital,sub-region,region,activity_class,years_since_creation,amount_capital,personality_type
0,332224393,1,Société à responsabilité limitée,F,EUR,Nord,Hauts-de-France,Industries alimentaires,34.0,7622.45,0
1,798102158,0,Société à responsabilité limitée,F,EUR,Paris,Île-de-France,"Commerce de détail, à l'exception des automobi...",6.0,10000.00,0
2,414305508,1,Société à responsabilité limitée,F,EUR,Seine-et-Marne,Île-de-France,"Commerce de détail, à l'exception des automobi...",22.0,7623.00,0
3,533910931,1,Société par actions simplifiée,F,EUR,Isère,Auvergne-Rhône-Alpes,"Commerce de gros, à l'exception des automobile...",8.0,4400000.00,0
4,809944416,0,Société par actions simplifiée,F,EUR,Ille-et-Vilaine,Bretagne,"Commerce de détail, à l'exception des automobi...",4.0,5000.00,0


In [85]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3573 entries, 0 to 3572
Data columns (total 11 columns):
siren                   3573 non-null object
employer                3573 non-null int64
legal_form              3558 non-null object
type_capital            3551 non-null object
ccy_capital             3068 non-null object
sub-region              3550 non-null object
region                  3550 non-null object
activity_class          3072 non-null object
years_since_creation    3545 non-null float64
amount_capital          3068 non-null float64
personality_type        3573 non-null int64
dtypes: float64(2), int64(2), object(7)
memory usage: 335.0+ KB


# Filter out irrelevant SIRENs

In [86]:
domains = pd.read_csv('/project/0_cleaning/output_cleaning/sirens_final.csv') 
labels = pd.read_csv('/project/1_feature_extraction/output_feature_extraction/labels.csv')
domains['siren'] = domains['siren'].astype(str)
labels['siren'] = labels['siren'].astype(str)

In [87]:
features_df_copy = features_df.copy()
features_df_copy = features_df_copy.merge(domains, how='left')
features_df_copy = features_df_copy.merge(labels, how='left')
features_df_copy.head()

,siren,employer,legal_form,type_capital,ccy_capital,sub-region,region,activity_class,years_since_creation,amount_capital,personality_type,domain,label
0,332224393,1,Société à responsabilité limitée,F,EUR,Nord,Hauts-de-France,Industries alimentaires,34.0,7622.45,0,www.kalifrais.fr,0.0
1,798102158,0,Société à responsabilité limitée,F,EUR,Paris,Île-de-France,"Commerce de détail, à l'exception des automobi...",6.0,10000.00,0,urbanwine.fr,0.0
2,414305508,1,Société à responsabilité limitée,F,EUR,Seine-et-Marne,Île-de-France,"Commerce de détail, à l'exception des automobi...",22.0,7623.00,0,www.zawai.fr,0.0
3,533910931,1,Société par actions simplifiée,F,EUR,Isère,Auvergne-Rhône-Alpes,"Commerce de gros, à l'exception des automobile...",8.0,4400000.00,0,www.villaverde.fr,0.0
4,809944416,0,Société par actions simplifiée,F,EUR,Ille-et-Vilaine,Bretagne,"Commerce de détail, à l'exception des automobi...",4.0,5000.00,0,protype3d.fr,0.0


In [88]:
features_df_copy.to_excel('filter_SIREN.xlsx')

After exploration, we found no clear pattern enabling us to exclude SIRENs based on obviously erroneous features (e.g. absurd legal form).

# Extract features

In [89]:
features_df.to_csv('/project/1_feature_extraction/output_feature_extraction/features_from_db.csv', index=False)

In [90]:
features_df.head()

,siren,employer,legal_form,type_capital,ccy_capital,sub-region,region,activity_class,years_since_creation,amount_capital,personality_type
0,332224393,1,Société à responsabilité limitée,F,EUR,Nord,Hauts-de-France,Industries alimentaires,34.0,7622.45,0
1,798102158,0,Société à responsabilité limitée,F,EUR,Paris,Île-de-France,"Commerce de détail, à l'exception des automobi...",6.0,10000.00,0
2,414305508,1,Société à responsabilité limitée,F,EUR,Seine-et-Marne,Île-de-France,"Commerce de détail, à l'exception des automobi...",22.0,7623.00,0
3,533910931,1,Société par actions simplifiée,F,EUR,Isère,Auvergne-Rhône-Alpes,"Commerce de gros, à l'exception des automobile...",8.0,4400000.00,0
4,809944416,0,Société par actions simplifiée,F,EUR,Ille-et-Vilaine,Bretagne,"Commerce de détail, à l'exception des automobi...",4.0,5000.00,0


In [91]:
features_df.isna().sum()

siren                     0
employer                  0
legal_form               15
type_capital             22
ccy_capital             505
sub-region               23
region                   23
activity_class          501
years_since_creation     28
amount_capital          505
personality_type          0
dtype: int64